In [6]:
import xarray as xr

ds_pwat = xr.open_dataset("/home/scratch/peldridge/pwat_tot.nc")          # data var: "pwat"
ds_iso  = xr.open_dataset("/home/scratch/peldridge/hgt_tot.nc") # data var: "isobaricInhPa"


ds_pwat

<xarray.Dataset> Size: 2GB
Dimensions:                (time: 7305, latitude: 159, longitude: 359)
Coordinates:
    number                 int64 8B ...
  * time                   (time) datetime64[ns] 58kB 2000-01-03 ... 2019-12-30
    step                   timedelta64[ns] 8B ...
    atmosphereSingleLayer  float64 8B ...
  * latitude               (latitude) float64 1kB 55.0 54.75 54.5 ... 15.75 15.5
    valid_time             (time) datetime64[ns] 58kB ...
  * longitude              (longitude) float64 3kB -140.0 -139.8 ... -50.5
Data variables:
    pwat                   (time, latitude, longitude) float32 2GB ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          2
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    history:                 2026-01-30T22:03 GRIB to CDM+CF via cfgrib-0.9.1...

In [7]:
ds_iso

<xarray.Dataset> Size: 421MB
Dimensions:        (time: 7305, latitude: 80, longitude: 180)
Coordinates:
    number         int64 8B ...
  * time           (time) datetime64[ns] 58kB 2000-01-07 ... 2019-12-31
    step           timedelta64[ns] 8B ...
    isobaricInhPa  float64 8B ...
  * latitude       (latitude) float64 640B 55.0 54.5 54.0 ... 16.5 16.0 15.5
    valid_time     (time) datetime64[ns] 58kB ...
  * longitude      (longitude) float64 1kB -140.0 -139.5 -139.0 ... -51.0 -50.5
Data variables:
    gh             (time, latitude, longitude) float32 421MB ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          2
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    history:                 2026-01-30T20:05 GRIB to CDM+CF via cfgrib-0.9.1...

In [8]:
print("pwat lat:", ds_pwat.latitude.size, ds_pwat.latitude.values[:3], ds_pwat.latitude.values[-3:])
print("iso  lat:", ds_iso.latitude.size,  ds_iso.latitude.values[:3],  ds_iso.latitude.values[-3:])
print("pwat lon:", ds_pwat.longitude.size, ds_pwat.longitude.values[:3], ds_pwat.longitude.values[-3:])
print("iso  lon:", ds_iso.longitude.size,  ds_iso.longitude.values[:3],  ds_iso.longitude.values[-3:])

pwat lat: 159 [55.   54.75 54.5 ] [16.   15.75 15.5 ]
iso  lat: 80 [55.  54.5 54. ] [16.5 16.  15.5]
pwat lon: 359 [-140.   -139.75 -139.5 ] [-51.   -50.75 -50.5 ]
iso  lon: 180 [-140.  -139.5 -139. ] [-51.5 -51.  -50.5]


## pwat is at a 0.25 degree res, z500 is at a 0.5 degree res. Going to coarsen to input into the som

In [10]:
ds_pwat = ds_pwat.sortby("time")
ds_iso  = ds_iso.sortby("time")

ds_pwat_on_z500 = ds_pwat.reindex(
    latitude=ds_iso.latitude,
    longitude=ds_iso.longitude,
    method="nearest"
)

ds_out = xr.merge([ds_pwat_on_z500, ds_iso], join="exact")
ds_out = ds_out.assign_coords(longitude=(ds_out.longitude % 360)).sortby("longitude")

ds_out.to_netcdf("/home/scratch/dwefer/GEFSv12/z500_pwat.nc")